<a href="https://colab.research.google.com/github/tanielfranklin/IRPF/blob/main/IRPF_Status_Invest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importações

In [20]:
import pandas as pd

### Carregar dados

In [21]:
!git clone https://github.com/tanielfranklin/IRPF

fatal: destination path 'IRPF' already exists and is not an empty directory.


In [22]:
%cd /content/IRPF
!git pull https://github.com/tanielfranklin/IRPF
%cd ..

/content/IRPF
From https://github.com/tanielfranklin/IRPF
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
/content


### Preparação do arquivo

In [23]:
df = pd.read_excel ('/content/IRPF/carteira-export.xlsx')
df=df.apply(lambda x: x.str.replace('.','')) # remover ponto de mil
df=df.apply(lambda x: x.str.replace(',','.')) # remover ponto de mil
df=df.apply(lambda x: x.str.replace('Fundos imobiliários','FII')) # FII
df=df.apply(lambda x: x.str.replace('CLEAR CORRETORA','Clear')) # Clear
df=df.apply(lambda x: x.str.replace('XP INVESTIMENTOS CCTVM S/A','XP')) # Clea
df.insert(0,'Data',pd.to_datetime(df['Data operação'], format='%d/%m/%Y'),True)
df.insert(6,'Preço',df['Preço unitário'].astype(float),True)
df.insert(7,'Qtd',df['Quantidade'].astype(float),True)
df.insert(1,'Ano',pd.DatetimeIndex(df['Data']).year,True)
df.insert(1,'Mes',pd.DatetimeIndex(df['Data']).month,True)
df.insert(1,'Dia',pd.DatetimeIndex(df['Data']).day,True)
df.drop(["Data operação","Preço unitário","Quantidade","Corretagem","Taxas","Impostos","IRRF"], axis=1, inplace=True)
df.rename(columns={'Operação C/V':'C/V'}, inplace=True)
df.rename(columns={'Código Ativo':'Ativo'}, inplace=True)
df.rename(columns={'Categoria':'cat'}, inplace=True)
df.head(n=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,Data,Dia,Mes,Ano,cat,Ativo,C/V,Preço,Qtd,Corretora
0,2022-03-18,18,3,2022,Ações,CSAN3,C,22.16,10.0,Clear
1,2022-03-16,16,3,2022,Ações,SULA4,V,11.10,100.0,Clear
2,2022-03-16,16,3,2022,Ações,PRIO3,C,23.22,50.0,Clear
3,2022-03-16,16,3,2022,FII,BRCR11,C,66.96,6.0,XP
4,2022-03-11,11,3,2022,Ações,CSAN3,C,21.88,18.0,Clear


## Definindo o Ano base

In [ ]:
data_base='2021-12-31'

### Ajuste Arquivo para ano base

In [24]:

df2021 = df.loc[(df['Data'] <= data_base )]
cond=df2021['C/V']=='C'
df2021.loc[cond,'C/V'] = 1.0
cond=df2021['C/V']=='V'
df2021.loc[cond,'C/V'] = -1.0
df2021.loc[:,'Qtd']=df2021['Qtd']*df2021['C/V']
df2021.insert(9,'custo',df2021['Preço']*df2021['Qtd'],True)
#df2021.insert(7,'PM',df2021['custo']/df2021['Quantidade'],True)

df2021A=df2021.loc[(df['cat'] == 'Ações')| (df['cat'] == 'FII')]
df2021A

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Data,Dia,Mes,Ano,cat,Ativo,C/V,Preço,Qtd,custo,Corretora
28,2021-12-30,30,12,2021,FII,XPML11,1.0,97.66,6.0,585.96,XP
29,2021-12-30,30,12,2021,Ações,BBAS3,1.0,28.95,100.0,2895.0,Clear
30,2021-12-30,30,12,2021,Ações,ITSA4,1.0,8.94,100.0,894.0,Clear
31,2021-12-30,30,12,2021,Ações,CPLE3,1.0,6.07,100.0,607.0,Clear
32,2021-12-30,30,12,2021,FII,VISC11,1.0,103.52,6.0,621.12,XP
...,...,...,...,...,...,...,...,...,...,...,...
211,2018-12-04,4,12,2018,Ações,ITSA4,-1.0,11.93,-100.0,-1193.0,Clear
212,2018-11-16,16,11,2018,Ações,ITSA4,1.0,11.74,100.0,1174.0,Clear
213,2018-09-19,19,9,2018,Ações,ITSA4,-1.0,9.78,-100.0,-978.0,Clear
214,2018-09-18,18,9,2018,Ações,ITSA4,1.0,9.95,100.0,995.0,Clear


## Relatório Ações ano base

### Bens e direitos

In [25]:
DFPM=df2021A.groupby(['cat','Ativo','Corretora'])['Qtd','custo'].sum()
DFPM.insert(2,'PM',DFPM['custo']/DFPM['Qtd'],True)
DFPM=DFPM.reset_index()  
DFPM
print(f"Relatório Bens e direitos {data_base}")
for i in DFPM.loc[(DFPM["cat"]=="Ações")].index: 
    print(f"Ações de {DFPM['Ativo'][i]} - Qtd: {int(DFPM['Qtd'][i])} a um custo médio de R${DFPM['PM'][i]:.2f} e custo total de R${DFPM['custo'][i]:.2f} custodiados na Corretora {DFPM['Corretora'][i]}")
    print("-----------------------------------------------------------------------------")

Relatório Bens e direitos 2021-12-31
Ações de AESB3 - Qtd: 326 a um custo médio de R$14.37 e custo total de R$4684.20 custodiados na Corretora Clear
-----------------------------------------------------------------------------
Ações de ALUP11 - Qtd: 140 a um custo médio de R$27.73 e custo total de R$3882.60 custodiados na Corretora Clear
-----------------------------------------------------------------------------
Ações de BBAS3 - Qtd: 100 a um custo médio de R$28.95 e custo total de R$2895.00 custodiados na Corretora Clear
-----------------------------------------------------------------------------
Ações de BBSE3 - Qtd: 135 a um custo médio de R$28.04 e custo total de R$3785.89 custodiados na Corretora Clear
-----------------------------------------------------------------------------
Ações de CPLE3 - Qtd: 100 a um custo médio de R$6.07 e custo total de R$607.00 custodiados na Corretora Clear
-----------------------------------------------------------------------------
Ações de CPLE6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [82]:
print(f'Relatório Bens e direitos {data_base} - Total de itens: {len(DFPM.loc[(DFPM["cat"]=="FII")].index)}')
print()
for i in DFPM.loc[(DFPM["cat"]=="FII")].index: 
    print(f"FII: Cotas de {DFPM['Ativo'][i]} - Qtd: {int(DFPM['Qtd'][i])} a um custo médio de R${DFPM['PM'][i]:.2f} e custo total de R${DFPM['custo'][i]:.2f} custodiados na Corretora {DFPM['Corretora'][i]}")
    print("-----------------------------------------------------------------------------")

Relatório Bens e direitos 2021-12-31 - Total de itens: 19

FII: Cotas de ALZR11 - Qtd: 22 a um custo médio de R$124.93 e custo total de R$2748.50 custodiados na Corretora XP
-----------------------------------------------------------------------------
FII: Cotas de BCFF11 - Qtd: 104 a um custo médio de R$87.06 e custo total de R$9054.14 custodiados na Corretora XP
-----------------------------------------------------------------------------
FII: Cotas de BRCR11 - Qtd: 50 a um custo médio de R$84.13 e custo total de R$4206.30 custodiados na Corretora XP
-----------------------------------------------------------------------------
FII: Cotas de GGRC11 - Qtd: 45 a um custo médio de R$120.64 e custo total de R$5428.97 custodiados na Corretora XP
-----------------------------------------------------------------------------
FII: Cotas de HABT11 - Qtd: 37 a um custo médio de R$116.58 e custo total de R$4313.59 custodiados na Corretora XP
-------------------------------------------------------

### Vendas

In [27]:
df_vendas=df2021A.loc[(df2021A['C/V'] == -1.0)]
df_vendas


,Data,Dia,Mes,Ano,cat,Ativo,C/V,Preço,Qtd,custo,Corretora
34,2021-12-09,9,12,2021,Ações,PETR4,-1.0,29.35,-100.0,-2935.0,Clear
89,2021-08-18,18,8,2021,FII,VISC11,-1.0,103.50,-10.0,-1035.0,XP
209,2019-01-21,21,1,2019,Ações,ITSA4,-1.0,12.83,-100.0,-1283.0,Clear
211,2018-12-04,4,12,2018,Ações,ITSA4,-1.0,11.93,-100.0,-1193.0,Clear
213,2018-09-19,19,9,2018,Ações,ITSA4,-1.0,9.78,-100.0,-978.0,Clear


In [96]:
def get_cost_date(ticker,dset,date):
    PM=dset.loc[(df['Data'] < date)&(df['Ativo'] == ticker)&(df['C/V'] == 'C')]
    #print(PM)
    medio=PM.groupby(['Ativo'])['custo'].sum()/PM.groupby(['Ativo'])['Qtd'].sum()
    custo=PM.groupby(['Ativo'])['custo'].sum()
    #print(custo,medio)
    #return custo[0],medio[0]
    return custo[0],medio[0]


In [95]:
def report_vendas_acoes(ano):
    mes=["janeiro", "fevereiro", "março", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro", "novembro" ,"dezembro"]
    df_vendas=df2021A.loc[(df2021A['C/V'] == -1.0)&(df2021A['Ano'] == ano)]
    print("############VENDAS##############")
    for index,i in enumerate(mes):
        print("")
        print(f"-------------{i}---------------")
        #print("")
        for j,k in zip(df_vendas['Mes'].values,df_vendas['Mes'].index):
            if j==index+1:
                #print(j,k)
                cost,PM=get_cost_date(df_vendas.loc[k]['Ativo'],df2021A,df_vendas.loc[k]['Data'])
                print(f"-Venda de {df_vendas.loc[k]['Ativo']} a R${df_vendas.loc[k]['Preço']} qtd: {-1*int(df_vendas.loc[k]['Qtd'])}"\
                      f" Lucro=({-df_vendas.loc[k]['custo']:.2f} - {-1*int(df_vendas.loc[k]['Qtd'])}*{PM:.2f})"\
                      f"= {-1*df_vendas.loc[k]['custo']-(-1*int(df_vendas.loc[k]['Qtd'])*PM):.2f}")

report_vendas_acoes(2021)


############VENDAS##############

-------------janeiro---------------

-------------fevereiro---------------

-------------março---------------

-------------abril---------------

-------------maio---------------

-------------junho---------------

-------------julho---------------

-------------agosto---------------
-Venda de VISC11 a R$103.5 qtd: 10 Lucro=(1035.00 - 10*112.54)= -90.42

-------------setembro---------------

-------------outubro---------------

-------------novembro---------------

-------------dezembro---------------
-Venda de PETR4 a R$29.35 qtd: 100 Lucro=(2935.00 - 100*22.05)= 730.00
